In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numba
import pathlib
import time
import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling
import pathlib
from tqdm.notebook import tqdm
import cv2
import gc
from skimage.measure import label, regionprops
from skimage.morphology import disk


In [ ]:
import numpy as np
from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input,Dropout,ZeroPadding2D
from keras.layers import BatchNormalization
from keras.layers import Conv2D,Flatten,Conv2DTranspose,UpSampling2D,Concatenate
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras import backend as K
from keras.applications import DenseNet121 as DenseNet121Keras
import time
import tensorflow as tf

#DenseNet paper https://arxiv.org/pdf/1608.06993.pdf
def transition_block(x, reduction, name):
    """A Transition block.
    A block which joins 2 dense-blocks, reduction
    factor reduces the total number of feature maps
    to be passed from DenseBlock 1 to 2
    This model is called - DenseNet-C (compression = 0.5)

    # Arguments
        x: input tensor.
        reduction: float, compression rate at transition layers.
        name: string, block label.

    # Returns
        output tensor for the block.
    """
    bn_axis =  3 if K.image_data_format() == 'channels_last' else 1
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(K.int_shape(x)[bn_axis] * reduction), 1,
                      use_bias=False,kernel_initializer = 'he_normal',
                      name=name + '_conv')(x)
    x = AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def conv_block(x, growth_rate, name):
    """A building block for a dense block.

    As per paper as dense block is having  X number of Conv blocks
    Every Conv block will have [1x1-Conv then  3x3-Conv]
    Every convolution   having BN+RELU prior to it

    # Arguments
        x: input tensor.
        growth_rate: float, growth rate at dense layers.
        name: string, block label.

    # Returns
        Output tensor for the block.
    """
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    # 1x1-Convolution
    x1 = BatchNormalization(axis=bn_axis,
                            epsilon=1.001e-5,
                            name=name + '_0_bn')(x)
    x1 = Activation('relu', name=name + '_0_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1,
                use_bias=False,kernel_initializer = 'he_normal',
                name=name + '_1_conv')(x1)

    # 3x3-Convolution
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                   name=name + '_1_bn')(x1)
    x1 = Activation('relu', name=name + '_1_relu')(x1)
    x1 = Conv2D(growth_rate, 3,
                padding='same',
                use_bias=False,kernel_initializer = 'he_normal',
                name=name + '_2_conv')(x1)
    x = Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
    return x

def dense_block(x, blocks, name):
    """A dense block.
    This block has - blocks number of Conv_blocks
    # Arguments
        x: input tensor.
        blocks: integer, the number of building blocks.
        name: string, block label.

    # Returns
        output tensor for the block.
    """
    for i in range(blocks):
        x = conv_block(x, growth_rate=32, name=name + '_block' + str(i + 1))
    return x


def DenseNet(inputShape=(256, 256, 3), nClasses=2,
                     pooling='avg',blocks = [6, 12, 24, 16],
                     include_top=False):
    img_input = Input(shape=inputShape)

    with tf.name_scope('EntryBlock') as scope:
        # define initial block
        bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
        x = ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
        x = Conv2D(64, 7, strides=2, use_bias=False, kernel_initializer='he_normal', name='conv1/conv')(x)
        x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(x)
        x = Activation('relu', name='conv1/relu')(x)
        x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
        x = MaxPooling2D(3, strides=2, name='pool1')(x)

    with tf.name_scope('DenseBlock-1') as scope:
        # denseblock1
        x = dense_block(x, blocks[0], name='DB-1')
        x = transition_block(x, 0.5, name='TB-1')

    with tf.name_scope('DenseBlock-2') as scope:
        # denseblock2
        x = dense_block(x, blocks[1], name='DB-2')
        x = transition_block(x, 0.5, name='TB-2')

    with tf.name_scope('DenseBlock-3') as scope:
        # denseblock3
        x = dense_block(x, blocks[2], name='DB-3')
        x = transition_block(x, 0.5, name='TB-3')

    with tf.name_scope('DenseBlock-4') as scope:
        # denseblock4
        x = dense_block(x, blocks[3], name='DB-4')
        x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
        x = Activation('relu', name='relu')(x)

    with tf.name_scope('FC') as scope:
        if include_top:
            x = GlobalAveragePooling2D(name='avg_pool')(x)
            act = 'sigmoid' if nClasses == 1 else 'softmax'
            x = Dense(nClasses, activation=act, name='fc')(x)
        else:
            if pooling == 'avg':
                x = GlobalAveragePooling2D(name='avg_pool')(x)
            elif pooling == 'max':
                x = GlobalMaxPooling2D(name='max_pool')(x)

    model = Model(inputs=img_input, outputs=x)

    return model

def DenseNet121(inputShape=(256, 256, 3), nClasses=2,
                     pooling='avg',
                     include_top=False):

    return DenseNet(inputShape=(256, 256, 3), nClasses=2,
                     pooling='avg',blocks = [6, 12, 24, 16],
                     include_top=False)

def DenseNet169(inputShape=(256, 256, 3), nClasses=2,
                     pooling='avg',
                     include_top=False):

    return DenseNet(inputShape=(256, 256, 3), nClasses=2,
                     pooling='avg',blocks = [6, 12, 32, 32],
                     include_top=False)

def DenseNet201(inputShape=(256, 256, 3), nClasses=2,
                     pooling='avg',
                     include_top=False):

    return DenseNet(inputShape=(256, 256, 3), nClasses=2,
                     pooling='avg',blocks = [6, 12, 48, 32],
                     include_top=False)


def encoder_up_sample_block(input,nFilters,level,
                  kernel_size=2,
                  stride = 2,
                  useConv2DTranspose = False):

    if useConv2DTranspose:
        x = Conv2DTranspose(filters=nFilters, kernel_size=kernel_size, strides=stride,
                            name='Encoder_Up_' + str(level) + '_conv2dTrans_' + str(level))(input)
    else:
        x = UpSampling2D(size=kernel_size, interpolation='bilinear',name='Encoder_Up_' + str(level) + '_Up_' + str(level))(input)
    return x

def conv_bn_act_block(x,
                  nfilters = 64,
                  num_conv_blocks=2,
                  level = 0,
                  kernel_size = 3,
                  pad = 'same',
                  conv_stride = 1,
                  activation='relu',
                  drop_out = True):

    for i in range(num_conv_blocks):
        x = Conv2D(filters=nfilters, kernel_size=kernel_size, strides=conv_stride, padding=pad,
                   kernel_initializer='he_normal', name='Encoder_Level_{}_Conv_{}'.format(level,i+1))(x)
        x = BatchNormalization(name='Encoder_Level_{}_BN_{}'.format(level,i+1))(x)
        x = Activation(activation, name='Encoder_Level_{}_Act_{}'.format(level,i+1))(x)

    # x = Conv2D(filters=nfilters, kernel_size=kernel_size, strides=conv_stride, padding=pad,
    #            kernel_initializer='he_normal', name='Encoder_Level_{}_Conv_b'.format(level))(x)
    # x = BatchNormalization(name='Encoder_Level_{}_BN_b'.format(level))(x)
    # x = Activation(activation, name='Encoder_Level_{}_Act_b'.format(level))(x)

    if drop_out:
        x = Dropout(0.5)(x)

    return x

def DenseNet121_UNET(inputShape=(256,256,3),nClasses=2,
                     pooling = 'avg',
                     drop_out = False,
                     reduce_encoder_featuremap_in_decoder = False,
                     reduction_factor = 0.5,
                     include_top=False):

    blocks = [6,12,24,16]
    img_input = Input(shape=inputShape)

    #define initial block
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    x = ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=False, kernel_initializer = 'he_normal',name='conv1/conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(x)
    x = Activation('relu', name='conv1/relu')(x)
    conv1_unet = x
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D(3, strides=2, name='pool1')(x)

    with tf.name_scope('DenseBlock-1') as scope:
        # denseblock1
        x = dense_block(x,blocks[0],name='DB-1')
        conv2_unet = x
        x = transition_block(x,0.5,name='TB-1')

    with tf.name_scope('DenseBlock-2') as scope:
        # denseblock2
        x = dense_block(x, blocks[1], name='DB-2')
        conv3_unet = x
        x = transition_block(x, 0.5, name='TB-2')

    with tf.name_scope('DenseBlock-3') as scope:
        # denseblock3
        x = dense_block(x, blocks[2], name='DB-3')
        conv4_unet = x
        x = transition_block(x, 0.5, name='TB-3')
    with tf.name_scope('DenseBlock-4') as scope:
        # denseblock4
        x = dense_block(x, blocks[3], name='DB-4')
        x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
        conv5_unet = x

    ''' Encoder '''
    # up-level 1
    nFilters = 512
    level =1
    with tf.name_scope('Encoder-Up-1') as scope:
        up_out_1 = encoder_up_sample_block(conv5_unet, nFilters=nFilters, level=level, useConv2DTranspose=False)
        up_out_1 = conv_bn_act_block(up_out_1, nfilters=nFilters, level='{}_1'.format(level), kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)
        if reduce_encoder_featuremap_in_decoder:
            reqfliters = int(K.int_shape(conv4_unet)[bn_axis] * reduction_factor)
            conv4_unet = conv_bn_act_block(conv4_unet, nfilters=reqfliters, num_conv_blocks=1,level='{}_reduce_enc_1x1'.format(level), kernel_size=1,
                                         pad='same', conv_stride=1, activation='relu', drop_out=drop_out)

        up_out_1 = Concatenate(axis=-1)([up_out_1,conv4_unet])
        up_out_1 = conv_bn_act_block(up_out_1, nfilters = nFilters,level ='{}_2'.format(level),kernel_size = 3,
                      pad = 'same',conv_stride = 1,activation='relu',drop_out = drop_out)

    with tf.name_scope('Encoder-Up-2') as scope:
        # up-level 2
        nFilters = nFilters // 2 #256
        level = level+1
        up_out_2 = encoder_up_sample_block(up_out_1, nFilters=nFilters, level=level, useConv2DTranspose=False)
        up_out_2 = conv_bn_act_block(up_out_2, nfilters=nFilters, level='{}_1'.format(level), kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)
        if reduce_encoder_featuremap_in_decoder:
            reqfliters = int(K.int_shape(conv3_unet)[bn_axis] * reduction_factor)
            conv3_unet = conv_bn_act_block(conv3_unet, nfilters=reqfliters, num_conv_blocks=1,level='{}_reduce_enc_1x1'.format(level), kernel_size=1,
                                         pad='same', conv_stride=1, activation='relu', drop_out=drop_out)

        up_out_2 = Concatenate(axis=-1)([up_out_2, conv3_unet])
        up_out_2 = conv_bn_act_block(up_out_2, nfilters=nFilters, level='{}_2'.format(level), kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)

    with tf.name_scope('Encoder-Up-3') as scope:
        # up-level 3
        nFilters = nFilters // 2 #128
        level = level + 1
        up_out_3 = encoder_up_sample_block(up_out_2, nFilters=nFilters, level=level, useConv2DTranspose=False)
        up_out_3 = conv_bn_act_block(up_out_3, nfilters=nFilters, level='{}_1'.format(level), kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)
        if reduce_encoder_featuremap_in_decoder:
            reqfliters = int(K.int_shape(conv2_unet)[bn_axis] * reduction_factor)
            conv2_unet = conv_bn_act_block(conv2_unet, nfilters=reqfliters, num_conv_blocks=1,level='{}_reduce_enc_1x1'.format(level), kernel_size=1,
                                         pad='same', conv_stride=1, activation='relu', drop_out=drop_out)
        up_out_3 = Concatenate(axis=-1)([up_out_3, conv2_unet])
        up_out_3 = conv_bn_act_block(up_out_3, nfilters=nFilters, level='{}_2'.format(level), kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)

    with tf.name_scope('Encoder-Up-4') as scope:
        # up-level 4
        nFilters = nFilters // 2 #64
        level = level + 1
        up_out_4 = encoder_up_sample_block(up_out_3, nFilters=nFilters, level=level, useConv2DTranspose=False)
        up_out_4 = conv_bn_act_block(up_out_4, nfilters=nFilters, level='{}_1'.format(level), kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)
        up_out_4 = Concatenate(axis=-1)([up_out_4, conv1_unet])
        up_out_4 = conv_bn_act_block(up_out_4, nfilters=nFilters, level='{}_2'.format(level), kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)

    with tf.name_scope('Encoder-Up-5') as scope:
        level = level + 1
        up_out_5 = encoder_up_sample_block(up_out_4, nFilters=nFilters, level=level, useConv2DTranspose=False)
        up_out_5 = conv_bn_act_block(up_out_5, nfilters=nFilters, level=level, kernel_size=3,
                                     pad='same', conv_stride=1, activation='relu', drop_out=drop_out)

    with tf.name_scope('FC') as scope:
        act = 'sigmoid' if nClasses == 1 else 'softmax'
        out = Conv2D(nClasses,kernel_size=1, strides=1, padding='same',activation=act,name='final_conv1x1_{}'.format(act))(up_out_5)

    model = Model(inputs=img_input,outputs=out)

    return model

if False:
    model = DenseNet121_UNET(inputShape=(512,512,3),nClasses=1,
                             drop_out=True,
                             reduce_encoder_featuremap_in_decoder=True,
                             reduction_factor=0.5,
                             include_top=False)
    model.summary()
    
    print('loading weight from pretrained..')
    modelKeras = DenseNet121Keras(input_shape=(512,512,3),include_top = False,weights='imagenet')
    modelKeras.summary()
    ts = time.time()
    for i in range(2,len(modelKeras.layers)):
        model.layers[i].set_weights(modelKeras.layers[i].get_weights())
    print('Time taken to load and assign weights-',time.time()-ts)
    model.summary()
    print('Done')

In [ ]:
@numba.njit()
def rle_numba(pixels):
    size = len(pixels)
    points = []
    if pixels[0] == 1: points.append(0)
    flag = True
    for i in range(1, size):
        if pixels[i] != pixels[i-1]:
            if flag:
                points.append(i+1)
                flag = False
            else:
                points.append(i+1 - points[-1])
                flag = True
    if pixels[-1] == 1: points.append(size-points[-1]+1)    
    return points

def rle_numba_encode(image,img_size_hw):
    if image.shape[0]!=shape[0] or image.shape[1] !=shape[1]:
        image = cv2.resize(image,(img_size_hw[1],img_size_hw[0]),interpolation=cv2.INTER_AREA)
    pixels = image.flatten(order = 'F')
    points = rle_numba(pixels)
    return ' '.join(str(x) for x in points)

def rle_encode_less_memory(img,shape):
    ts = time.time()
    if img.shape[0]!=shape[0] or img.shape[1] !=shape[1]:
        print('resizing-rle generation')
        img = cv2.resize(img,(shape[1],shape[0]),interpolation=cv2.INTER_AREA)
        
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    res = ' '.join(str(x) for x in runs)
    print('time for RLE conversion-',time.time()-ts)
    return res

identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

In [ ]:
def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)

    x1[-1] = x - window

    x2 = (x1 + window).clip(0, x)

    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)

    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

In [ ]:
def get_tissue_mask(img,method='std',stdthd=20): #earlier 5
    #need rgb for std
    if method=='std':
        mask = np.std(img,axis=-1) >stdthd
    else:
        #otsu
        if np.ndim(img)==3:
            img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        thd = threshold_otsu(img)
        mask = img < thd
    return mask

def check_boundary_limit(yxpairArray,imgH,imgW):
    mask = np.logical_and(yxpairArray[:,1] < imgH,yxpairArray[:,3]<imgW) #check H and W not going beyond image dims
    #selectrow = mask.all(axis=1)
    return yxpairArray[mask]
    
def filterPoints(imgpath,point_sampling_level=4,patch_size=512,stride=448,min_percent_tissue_area=0.1):
    iodataset = rasterio.open(imgpath, transform = identity)
    x1,y1=0,0
    x2,y2 = iodataset.height, iodataset.width
    imgH, imgW = iodataset.height, iodataset.width
    factor = 2 ** point_sampling_level
    if True:
        wsiImage = iodataset.read([1,2,3],window=Window.from_slices((x1,x2),(y1,y2)))
        wsiImage = np.moveaxis(wsiImage, 0, -1)
        rescaled = (wsiImage.shape[1] // factor, wsiImage.shape[0] // factor)
        wsiImage_rsz = cv2.resize(wsiImage, rescaled, interpolation=cv2.INTER_LINEAR)
        #wsiImage_gray = cv2.cvtColor(self.wsiImage, cv2.COLOR_RGB2GRAY) 
    else:
        wsiImage_rsz = iodataset.read(
                            out_shape=(dataset.count,
                                int(dataset.height/factor),
                                int(dataset.width/factor)),resampling=Resampling.bilinear)
        wsiImage_rsz = np.moveaxis(wsiImage_rsz, 0, -1)
        
    valid_region_mask = get_tissue_mask(wsiImage_rsz,method='std')
    H, W = valid_region_mask.shape
    stride_mask_level = stride // factor
    patch_mask_level = patch_size // factor
    if True:
        overlap_level = patch_mask_level - stride_mask_level
        locations = make_grid(shape=(H,W),window=patch_mask_level,min_overlap=overlap_level )
        area_thd = min_percent_tissue_area * patch_mask_level * patch_mask_level
        yxpairList = []
        for i in range(locations.shape[0]):
            x1,x2,y1,y2 = np.int32(locations[i])
            if np.sum(valid_region_mask[x1:x2, y1: y2]) > area_thd:
                yxpairList.append([x1*factor,x2*factor,y1*factor, y2 * factor])
            
        del wsiImage_rsz
        del valid_region_mask
        del wsiImage
        del locations
    return np.reshape(yxpairList,[len(yxpairList),4])

In [ ]:
def postprocessing(img):
    from skimage.morphology import disk
    kernel = disk(3)
    #remove unwanted small detection
    #self.stitch_image = cv2.erode(self.stitch_image,kernel,iterations=1)
    #remove small white object- opening  ; Closing fills the hole in white image
    stitch_image = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel,iterations=1)
    return stitch_image

def processMask(maskFile,factor,fixThd=50):
    H,W = maskFile.shape
    resized = (W//factor,H//factor)
    resized_mask = cv2.resize(maskFile,resized,interpolation=cv2.INTER_AREA)

    kernel = disk(2)
    resized_mask = cv2.morphologyEx(resized_mask, cv2.MORPH_OPEN, kernel, iterations=3)
    #resized_mask = resized_mask >0
    labelImg = label(resized_mask)
    props = regionprops(labelImg)
    filterImg = np.zeros_like(resized_mask)
    for prp in props:
        if prp.area  < 1 or prp.perimeter <1:
            continue
        if prp.area < fixThd: #original size = fixThd*factor (=8)
            continue

        mask = labelImg==prp.label
        filterImg[mask] = 255
        # id = prp.label
        # area = prp.area
        # solid = prp.solidity
        # circulaity = 4 * area/(max(prp.perimeter,1))**2
        # stats.append([id,area,solid,circulaity])

    orig_mask = cv2.resize(filterImg, (W,H), interpolation=cv2.INTER_AREA)
    del filterImg
    return orig_mask

In [ ]:
model_wight_path1 = '/kaggle/input/densenet121unet/weights-run1-19-val_acc-0.992-val_dice_loss-0.202.hdf5'
model_wight_path2 = '/kaggle/input/densenet121unerbl/run2_boundary_loss.hdf5'
model = DenseNet121_UNET(inputShape=(512,512,3),nClasses=1,
                             drop_out=True,
                             reduce_encoder_featuremap_in_decoder=True,
                             reduction_factor=0.5,
                             include_top=False)
model.load_weights(model_wight_path2)
model.summary()

In [ ]:
p = pathlib.Path('../input/hubmap-kidney-segmentation')
subm = {}
import time
WINDOW = 1024;#512
STRIDE = 512
MIN_OVERLAP = 256#128 # 1024-896 = 128
NEW_SIZE = 1024
THRESHOLD = 0.25
fold_models = [model] #train/aaa6a05c
for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                        total = len(list(p.glob('test/*.tiff')))):
    
    print(f'{i+1} Predicting {filename.stem}')
    ts = time.time()
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
    print('orig points',slices.shape)
    if False:
        slices = filterPoints(filename.as_posix(),point_sampling_level=4,
                              patch_size=WINDOW,stride=STRIDE,
                              min_percent_tissue_area=0.25)
        print('total points b4 boundry check-',slices.shape)
        slices = check_boundary_limit(slices,imgH=dataset.shape[0],imgW=dataset.shape[1])
        print('total points after boundry check-',slices.shape)
    preds = np.zeros(dataset.shape, dtype=np.uint8)
    #print(preds.shape)
    for (x1,x2,y1,y2) in tqdm(slices,total=slices.shape[0]):
        image = dataset.read([1,2,3],
                    window=Window.from_slices((x1,x2),(y1,y2)))
        image = np.moveaxis(image, 0, -1)
        image = cv2.resize(image, (NEW_SIZE, NEW_SIZE),interpolation = cv2.INTER_LINEAR)
        #normlize
        image = np.float32(image)/255.0
        #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = np.expand_dims(image, 0)
        
#         pred = None
#         for fold_model in fold_models:
#             if pred is None:
#                 pred = np.squeeze(fold_model.predict(image))
#             else:
#                 pred += np.squeeze(fold_model.predict(image))
        
#         pred = pred/len(fold_models)
       
        pred = np.squeeze(model.predict(image))
        #print('pred',np.min(pred),np.max(pred))
        #print('image',np.min(image),np.max(image))
        pred = cv2.resize(pred, (WINDOW, WINDOW))
        preds[x1:x2,y1:y2] += (pred > THRESHOLD).astype(np.uint8)
    
    preds = (preds > 0.5).astype(np.uint8)
    #preds = postprocessing(preds)
    preds = processMask(preds,factor=4,fixThd=1000)
    preds = (preds > 0.5).astype(np.uint8)
    if False:
        basename = os.path.basename(filename)
        first_name = basename.split('.')[0]
        cv2.imwrite(f'./{first_name}_mask.png',preds*255)
    subm[i] = {'id':filename.stem, 'predicted': rle_encode_less_memory(preds,shape=dataset.shape)}
    #print(np.sum(preds))
    print('Time taken',time.time()-ts)
    del preds
    gc.collect();

In [ ]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('submission.csv', index=False)
submission.head()